In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from os import listdir
import json
import networkx as nx

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('./drive', force_remount=True)

path_prefix = './drive/My Drive/CS48002-NetworkScience/'

Mounted at ./drive


In [51]:
df = pd.read_csv(path_prefix+'network_dataset.csv', index_col=0)

In [52]:
df.sample()

,App Id,All Permissions,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
229,com.kiragames.unblockmefree,"[{'permission': 'view Wi-Fi connections', 'typ...",Unblock Me FREE,Puzzle,4.5,701925.0,"50,000,000+",50000000.0,96519838,True,...,http://www.kiragames.com,support@kiragames.com,"Aug 6, 2010","May 19, 2021",Everyone,http://kiragames.com/privacy-policy.html,True,True,False,2021-06-16 05:40:01


In [63]:
def permissionSimilarity(id1, id2):
  row1=df[df["App Id"]==id1]
  row2=df[df["App Id"]==id2]

  perms1=list(row1["All Permissions"])[0]
  perms2=list(row2["All Permissions"])[0]

  perms1 = list(eval(perms1))
  perms2 = list(eval(perms2))

  common_perms = set()

  for per1 in perms1:
    for per2 in perms2:
      if dict(per1)["permission"]==dict(per2)["permission"]:
        common_perms.add(dict(per1)["permission"])
  
  if len(perms1)+ len(perms2) == 0:
    return 1
  
  return len(common_perms) / (((len(perms1)+len(perms2))/2))

In [ ]:
import csv
from google.colab import files

edgeList = []

for i, row1 in df.iterrows():
  for j,row2 in df.iterrows():
    if row1["App Id"] != row2["App Id"] and j > i:
      edgeWeight = permissionSimilarity(row1["App Id"], row2["App Id"])
      if edgeWeight > 0: 
        edge = ((row1["App Id"], row2["App Id"], edgeWeight))
        edgeList.append(edge)
  print(i)
  
with open(path_prefix+'network_edges.txt', 'w') as f: 
  write = csv.writer(f)
  write.writerows(edgeList)

files.download(path_prefix+"network_edges.txt") 

In [65]:
appNet = nx.Graph()
appNet.add_weighted_edges_from(edgeList)

In [66]:
import community.community_louvain as community_louvain

communities = community_louvain.best_partition(appNet)

In [67]:
uniq = []
for key in communities:
  if communities[key] not in uniq:
    uniq.append(communities[key])
print(uniq)

[2, 0, 1]


In [68]:
with open(path_prefix+'app_communities_layer1.txt', 'w') as f: 
  f.write(json.dumps(communities))

files.download(path_prefix+"app_communities_layer1.txt") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [108]:
def community_ad_analysis(community):
  ad_free = 0
  ad_supported = 0

  for key in communities:
    if communities[key] == community:
      app = df[df["App Id"]==key]
      isAdSupported = str(list(app["Ad Supported"])[0])
      if isAdSupported == "True":
        ad_supported+=1
      elif isAdSupported == "False":
        ad_free +=1
        
  return ad_free , ad_supported


In [111]:
def community_category_analysis(community):
  categories = {}

  for key in communities:
    if communities[key] == community:
      app = df[df["App Id"]==key]
      category = list(app["Category"])[0]
      if category  not in categories:
        categories[category] = 1
      else:
        categories[category] += 1

  return categories

In [ ]:
cat = {}

for comm in uniq:
  cat[comm] = dict()
  cat[comm]["Category Distribution"] = community_category_analysis(comm)
  cat[comm]["Ad Supportedness"] = community_ad_analysis(comm)

In [114]:
import json
with open('layer1.json', 'w') as fp:
    json.dump(cat, fp, indent=4)

In [116]:
files.download("layer1.json") 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>